# Tutorial 17 - Navier-Stokes

In [ ]:
import typing

In [ ]:
import dolfinx.fem
import dolfinx.fem.petsc
import dolfinx.io
import gmsh
import mpi4py.MPI
import multiphenicsx.fem.petsc
import numpy as np
import numpy.typing
import petsc4py.PETSc
import ufl
import viskex

In [ ]:
# import rbnicsx.backends

## 1. Mesh generation

In [ ]:
pre_step_length = 4.0
after_step_length = 14.0
pre_step_height = 3.0
after_step_height = 5.0
mesh_size = 2e-1

In [ ]:
gmsh.initialize()
gmsh.model.add("navier_stokes")

In [ ]:
p0 = gmsh.model.geo.addPoint(0.0, after_step_height - pre_step_height, 0.0, mesh_size)
p1 = gmsh.model.geo.addPoint(pre_step_length, after_step_height - pre_step_height, 0.0, mesh_size)
p2 = gmsh.model.geo.addPoint(pre_step_length, 0.0, 0.0, mesh_size)
p3 = gmsh.model.geo.addPoint(pre_step_length + after_step_length, 0.0, 0.0, mesh_size)
p4 = gmsh.model.geo.addPoint(pre_step_length + after_step_length, after_step_height, 0.0, mesh_size)
p5 = gmsh.model.geo.addPoint(0.0, after_step_height, 0.0, mesh_size)
l0 = gmsh.model.geo.addLine(p0, p1)
l1 = gmsh.model.geo.addLine(p1, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p4)
l4 = gmsh.model.geo.addLine(p4, p5)
l5 = gmsh.model.geo.addLine(p5, p0)
boundary = gmsh.model.geo.addCurveLoop([l0, l1, l2, l3, l4, l5])
domain = gmsh.model.geo.addPlaneSurface([boundary])

In [ ]:
gmsh.model.geo.synchronize()
gmsh.model.addPhysicalGroup(1, [l5], 1)
gmsh.model.addPhysicalGroup(1, [l0, l1, l2, l4], 2)
gmsh.model.addPhysicalGroup(2, [domain], 1)
gmsh.model.mesh.generate(2)

In [ ]:
mesh, subdomains, boundaries = dolfinx.io.gmshio.model_to_mesh(
    gmsh.model, comm=mpi4py.MPI.COMM_WORLD, rank=0, gdim=2)
gmsh.finalize()

In [ ]:
viskex.dolfinx.plot_mesh(mesh)

In [ ]:
viskex.dolfinx.plot_mesh_tags(mesh, subdomains, "subdomains")

In [ ]:
viskex.dolfinx.plot_mesh_tags(mesh, boundaries, "boundaries")

## 2. Problem definition

In [ ]:
class Problem(object):
    """Define a nonlinear problem, and solve it with SNES."""

    def __init__(self) -> None:
        # Define function space
        V = dolfinx.fem.functionspace(mesh, ("Lagrange", 2, (mesh.geometry.dim, )))
        Q = dolfinx.fem.functionspace(mesh, ("Lagrange", 1))
        self._VQ = (V, Q)
        # Define trial and test functions
        (v, q) = (ufl.TestFunction(V), ufl.TestFunction(Q))
        (du, dp) = (ufl.TrialFunction(V), ufl.TrialFunction(Q))
        # Define solution components
        (u, p) = (dolfinx.fem.Function(V), dolfinx.fem.Function(Q))
        self._solutions = (u, p)
        # Define residual block form of the problem
        residual = [
            (
                ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx + ufl.inner(ufl.grad(u) * u, v) * ufl.dx
                - ufl.inner(p, ufl.div(v)) * ufl.dx
            ),
            ufl.inner(ufl.div(u), q) * ufl.dx
        ]
        self._residual = residual
        self._residual_cpp = dolfinx.fem.form(residual)
        # Define jacobian block form of the problem
        jacobian = [
            [ufl.derivative(residual[0], u, du), ufl.derivative(residual[0], p, dp)],
            [ufl.derivative(residual[1], u, du), ufl.derivative(residual[1], p, dp)]
        ]
        self._jacobian = jacobian
        self._jacobian_cpp = dolfinx.fem.form(jacobian)
        # Define boundary conditions for the problem
        u_in = dolfinx.fem.Function(V)
        u_wall = dolfinx.fem.Function(V)
        facets_in = boundaries.indices[boundaries.values == 1]
        facets_wall = boundaries.indices[boundaries.values == 2]
        bdofs_V_in = dolfinx.fem.locate_dofs_topological(V, mesh.topology.dim - 1, facets_in)
        bdofs_V_wall = dolfinx.fem.locate_dofs_topological(V, mesh.topology.dim - 1, facets_wall)
        bc_in = dolfinx.fem.dirichletbc(u_in, bdofs_V_in)
        bc_wall = dolfinx.fem.dirichletbc(u_wall, bdofs_V_wall)
        bcs = [bc_in, bc_wall]
        self._u_in = u_in
        self._bcs = bcs

    @property
    def function_spaces(self) -> typing.Tuple[
        dolfinx.fem.FunctionSpace, dolfinx.fem.FunctionSpace
    ]:
        """Return the function spaces of the problem."""
        return self._VQ

    @property
    def solution(self) -> typing.Tuple[dolfinx.fem.Function, dolfinx.fem.Function]:
        """Return the components of the solution of the problem for the latest parameter value."""
        return self._solutions

    @property
    def residual_form(self) -> typing.List[ufl.Form]:  # type: ignore[no-any-unimported]
        """Return the residual block form of the problem."""
        return self._residual

    @property
    def jacobian_form(self) -> typing.List[typing.List[ufl.Form]]:  # type: ignore[no-any-unimported]
        """Return the jacobian block form of the problem."""
        return self._jacobian

    @property
    def boundary_conditions(self) -> typing.List[dolfinx.fem.DirichletBC]:
        """Return the boundary conditions for the problem."""
        return self._bcs

    def create_snes_solution(self) -> petsc4py.PETSc.Vec:  # type: ignore[no-any-unimported]
        """
        Create a petsc4py.PETSc.Vec to be passed to petsc4py.PETSc.SNES.solve.

        The returned vector will be initialized with the initial guesses provided in `self._solutions`,
        properly stacked together in a single block vector.
        """
        x = dolfinx.fem.petsc.create_vector_block(self._residual_cpp)
        with multiphenicsx.fem.petsc.BlockVecSubVectorWrapper(x, [self._VQ[0].dofmap, self._VQ[1].dofmap]) as x_wrapper:
            for x_wrapper_local, sub_solution in zip(x_wrapper, self._solutions):
                with sub_solution.vector.localForm() as sub_solution_local:
                    x_wrapper_local[:] = sub_solution_local
        return x

    def update_solutions(self, x: petsc4py.PETSc.Vec) -> None:  # type: ignore[no-any-unimported]
        """Update `self._solutions` with data in `x`."""
        x.ghostUpdate(addv=petsc4py.PETSc.InsertMode.INSERT, mode=petsc4py.PETSc.ScatterMode.FORWARD)
        with multiphenicsx.fem.petsc.BlockVecSubVectorWrapper(x, [self._VQ[0].dofmap, self._VQ[1].dofmap]) as x_wrapper:
            for x_wrapper_local, sub_solution in zip(x_wrapper, self._solutions):
                with sub_solution.vector.localForm() as sub_solution_local:
                    sub_solution_local[:] = x_wrapper_local

    def _assemble_residual(  # type: ignore[no-any-unimported]
        self, snes: petsc4py.PETSc.SNES, x: petsc4py.PETSc.Vec, residual_vec: petsc4py.PETSc.Vec
    ) -> None:
        """Assemble the residual."""
        self.update_solutions(x)
        with residual_vec.localForm() as residual_vec_local:
            residual_vec_local.set(0.0)
        dolfinx.fem.petsc.assemble_vector_block(  # type: ignore[misc]
            residual_vec, self._residual_cpp, self._jacobian_cpp, self._bcs, x0=x, scale=-1.0)

    def _assemble_jacobian(  # type: ignore[no-any-unimported]
        self, snes: petsc4py.PETSc.SNES, x: petsc4py.PETSc.Vec, jacobian_mat: petsc4py.PETSc.Mat,
        preconditioner_mat: petsc4py.PETSc.Mat
    ) -> None:
        """Assemble the jacobian."""
        jacobian_mat.zeroEntries()
        dolfinx.fem.petsc.assemble_matrix_block(  # type: ignore[misc]
            jacobian_mat, self._jacobian_cpp, self._bcs, diagonal=1.0)  # type: ignore[arg-type]
        jacobian_mat.assemble()

    def solve(self, mu: np.typing.NDArray[np.float64]) -> typing.Tuple[
            dolfinx.fem.Function, dolfinx.fem.Function]:
        """Assign the provided parameter value to boundary conditions and solve the problem."""
        self._u_in.interpolate(lambda x: (mu[0] * 1 / 2.25 * (x[1] - 2) * (5 - x[1]), 0 * x[0]))
        return self._solve()

    def _solve(self) -> typing.Tuple[dolfinx.fem.Function, dolfinx.fem.Function]:
        """Solve the nonlinear problem with SNES."""
        snes = petsc4py.PETSc.SNES().create(mesh.comm)
        snes.setTolerances(max_it=20)
        snes.getKSP().setType("preonly")
        snes.getKSP().getPC().setType("lu")
        snes.getKSP().getPC().setFactorSolverType("mumps")
        residual_vec = dolfinx.fem.petsc.create_vector_block(self._residual_cpp)
        snes.setFunction(self._assemble_residual, residual_vec)
        jacobian_mat = dolfinx.fem.petsc.create_matrix_block(self._jacobian_cpp)
        snes.setJacobian(self._assemble_jacobian, J=jacobian_mat, P=None)
        snes.setMonitor(lambda _, it, residual: print(it, residual))
        solution = self.create_snes_solution()
        snes.solve(None, solution)
        self.update_solutions(solution)
        residual_vec.destroy()
        jacobian_mat.destroy()
        solution.destroy()
        snes.destroy()
        return self._solutions

In [ ]:
problem = Problem()

In [ ]:
mu_solve = np.array([80.0])
(solution_u, solution_p) = problem.solve(mu_solve)

In [ ]:
viskex.dolfinx.plot_vector_field(solution_u, "high fidelity velocity", glyph_factor=1e-2)

In [ ]:
viskex.dolfinx.plot_scalar_field(solution_p, "high fidelity pressure")